In [11]:
def ols(y,x):
    model = api.OLS(y,x).fit()
    return model.resid

def get_10_1(data_):
    data = data_.copy()
    data[10] = data.iloc[:,-1] - data.iloc[:,0]
    return data

def get_t(a):
    a_mean = (pd.DataFrame(a.mean(),columns=['α'])*100).applymap(lambda x:'{:.2f}'.format(x))
    t = pd.DataFrame(st.ttest_1samp(a,0,nan_policy='omit')[0]).applymap(lambda x:'({:.2f})'.format(x))
    p = pd.DataFrame(st.ttest_1samp(a,0,nan_policy='omit')[1])
    p_ = p.astype('object').copy()
    p_[p>0.1] = ''
    p_[p<=0.1] = '*'
    p_[p<=0.05] = '**'
    p_[p<=0.01] = '***'
    p = p_.copy()
    for i in range(len(p)):
        a_mean.iloc[i] = a_mean.iloc[i].values + p.iloc[i].values
    
    index = range(len(a_mean))
    a_mean.index = index
    t.index = index
    
    result = pd.concat([a_mean,t],axis=1).T
    if result.shape[1] <10: 
        result.columns = ['L','2','3','4','H','H-L']
    else:
        result.columns = ['L','2','3','4','5','6','7','8','9','H','H-L']
    return result

In [12]:
#shift是因为要预测未来一个月的收益，所以要将这些数据往前推一个月
def get_ff(path):
    ff = pd.read_csv(path,index_col=0).shift(-1)
    ff = ff.iloc[-len(month_ret)-1:-1]
    ff.index = month_ret.index
    return ff.iloc[:,1:].T[::2].T

In [15]:
capm = get_ff('D:/jupyterfile/因子数据/因子收益率序列/FF3_month.csv')[['mkt_rf1']]
ff3 = get_ff('D:/jupyterfile/因子数据/因子收益率序列/FF3_month.csv')
cht4 = get_ff('D:/jupyterfile/因子数据/因子收益率序列/CARHART4_month.csv').iloc[:,:-1]
ff5 = get_ff('D:/jupyterfile/因子数据/因子收益率序列/FF5_month.csv')
ff6 = pd.concat([cht4,ff5[['rmw1','cma1']]],axis=1)

models = [capm,ff3,cht4,ff5,ff6]

In [16]:
groups_return = dict(zip(['Sigm'],group_return_eq))
group_return = groups_return['Sigm'].dropna()
#apply默认对列处理，通过取对应列做回归得到回归系数
result = pd.concat([get_t(get_10_1(group_return.iloc[:,:-1].apply(lambda y:ols(y,model.loc[y.index])))) for model in models])
result.index = ['α1','','α3','','α4','','α5','','α6','']
# result.to_excel('../result/控制模型.xlsx',sheet_name='2')

In [17]:
result

,L,2,3,4,5,6,7,8,9,H,H-L
α1,0.68,0.80,0.88,1.02*,1.10*,0.98,1.16*,1.07*,1.15*,1.03,0.35
,(1.14),(1.36),(1.47),(1.68),(1.77),(1.57),(1.84),(1.70),(1.77),(1.59),(1.51)
α3,0.58,0.71,0.78,0.93,0.99,0.87,1.07*,0.96,1.03,0.90,0.32
,(0.99),(1.21),(1.32),(1.54),(1.60),(1.40),(1.71),(1.53),(1.59),(1.40),(1.37)
α4,0.57,0.71,0.79,0.93,1.00,0.89,1.08*,0.98,1.06,0.92,0.35
,(0.96),(1.21),(1.32),(1.53),(1.62),(1.43),(1.73),(1.56),(1.64),(1.43),(1.52)
α5,0.33,0.47,0.54,0.71,0.77,0.68,0.86,0.81,0.91,0.79,0.46**
,(0.56),(0.80),(0.91),(1.18),(1.25),(1.10),(1.37),(1.28),(1.40),(1.22),(2.00)
α6,0.39,0.54,0.63,0.77,0.85,0.76,0.93,0.87,0.97,0.84,0.45*
,(0.67),(0.92),(1.05),(1.28),(1.38),(1.22),(1.49),(1.38),(1.50),(1.30),(1.93)
